In [1]:
import mariadb
import sys

In [8]:
#funciones auxiliares:
def reemplazar_none(tup):
    return tuple('No' if elem is None else elem for elem in tup)

In [2]:
#Crea la conexión a la base con todos los tramites de todos los ministerios
# origen
try:
    conn_destino = mariadb.connect(
        user="root",
        password="carnesoya02",
        host="localhost",
        port=3306,
        database="schema_name_2"
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn_destino.cursor()

In [51]:
#Mediante una consulta obtenemos todos los nombres de las instituciones que usaremos para crear
#las bases para las distintas instituciones.
instituciones=[]
cur.execute("select Institución, count(Institución) tramites from datos_modificados group by Institución order by tramites desc limit 30")
for j in cur:
    instituciones.append(j[0])

# Creación de las conexiones y tablas

In [10]:
# Diccionario para almacenar las conexiones y los cursores
conexiones = {}
cursores = {}
prueba=[]
# Consulta SQL para obtener los datos y los nombres de las columnas
consulta_columnas = "SHOW COLUMNS FROM datos_modificados;"

# Consulta SQL para crear la tabla
consulta_crear_tabla = "CREATE TABLE nueva_tabla ({})"

# Ejecuta la consulta para obtener los nombres de las columnas
cur.execute(consulta_columnas)

# Obtiene los nombres de las columnas
nombres_columnas = [columna[0] for columna in cur.fetchall()]

# Crea la lista de definiciones de columna para la consulta de creación de tabla
definiciones_columnas = [f"{nombre_columna} TEXT" for nombre_columna in nombres_columnas]

# Formatea la lista de definiciones de columna en una cadena separada por comas
definiciones_columnas_str = ", ".join(definiciones_columnas)

# Formatea la consulta de creación de tabla con las definiciones de columna
consulta_crear_tabla = consulta_crear_tabla.format(definiciones_columnas_str)
# Bucle para crear conexiones dinámicamente
for j in range(0,5):
    try:
#-----------Crea una conexión y almacénala en el diccionario--------
        conexiones[f'conexion_{j}'] = mariadb.connect(
            user="root",
            password="carnesoya02",
            host="localhost",  # Concatena el valor de j al nombre del host
            port=3313+j,  # Puerto específico de la base de datos en el equipo local
            database="schema_name"
        )
        print(f"Conexión {j} creada correctamente")
    except mariadb.Error as e:
        print(f"Error al conectar con la base de datos {j}: {e}")
    #--------------Creación de un cursor y guardarlo en un diccionario----------
    cursores[f'conexion_{j}']=conexiones[f'conexion_{j}'].cursor()
    #---------------------------------------------------------------------------
    #se seleccionan los tramites que están en una cierta institución
    cur.execute("select * from datos_modificados where Institución= %s",(instituciones[j],))
    datos = cur.fetchall()
    #Dejamos los datos en el formato requerido para insertarlos
    mi_tuple_modificado = tuple(map(reemplazar_none, datos))
    datos2=list(mi_tuple_modificado)
    # Ejecuta la consulta para crear la tabla
    cursores[f'conexion_{j}'].execute("CREATE TABLE nueva_tabla (Ministerio TEXT,Institucion TEXT,Codigo_Tramite TEXT,Que_tipo_de_producto_entrega_la_institucion_con_este_tramite TEXT,Nombre_del_tramite TEXT,Descripcion_del_tramite TEXT,Nivel_de_digitalizacion TEXT,Mediante_que_herramienta_digitalizo_el_tramite TEXT,Canal_digital TEXT,Canal_presencial TEXT,Canal_telefonico TEXT,Canal_moduloautoatencion TEXT,Url_institucional TEXT,Tiene_Ficha_ChileAtiende TEXT,Url_ficha_ChileAtiende TEXT,El_tramite_se_puede_realizar_todo_el_ano TEXT,Considera_mecanismo_de_autenticacion TEXT,Considera_pago TEXT,Detalles_de_pago TEXT,iniciativa TEXT,Marco_legal TEXT,Tipo_de_usuario_al_cual_va_dirigido_este_tramite TEXT,Tipo_de_persona_juridica TEXT,El_tramite_esta_focalizado_en_alguno_de_los_siguientes_grupos TEXT,El_tramite_requiere_Registro_Social_de_Hogares_RSH TEXT,Quien_puede_realizar_este_tramite TEXT,Requiere_de_algun_certificado TEXT,Detalle_certificados TEXT,Requiere_de_un_documento_notarial TEXT,Detalle_documentos TEXT,Finaliza_con_la_emision_de_un_certificado TEXT,involucra_firmaFEA TEXT,Tipo_de_Firma_Electronica_Avanzada_FEA TEXT);")    
    print("Tabla creada correctamente")

    # Consulta SQL para insertar los datos en la nueva tabla
    cursores[f'conexion_{j}'].executemany("INSERT INTO nueva_tabla (Ministerio, Institucion, Codigo_Tramite ,Que_tipo_de_producto_entrega_la_institucion_con_este_tramite ,Nombre_del_tramite ,Descripcion_del_tramite ,Nivel_de_digitalizacion ,Mediante_que_herramienta_digitalizo_el_tramite ,Canal_digital ,Canal_presencial ,Canal_telefonico ,Canal_moduloautoatencion ,Url_institucional ,Tiene_Ficha_ChileAtiende ,Url_ficha_ChileAtiende ,El_tramite_se_puede_realizar_todo_el_ano ,Considera_mecanismo_de_autenticacion ,Considera_pago ,Detalles_de_pago ,iniciativa ,Marco_legal ,Tipo_de_usuario_al_cual_va_dirigido_este_tramite ,Tipo_de_persona_juridica ,El_tramite_esta_focalizado_en_alguno_de_los_siguientes_grupos ,El_tramite_requiere_Registro_Social_de_Hogares_RSH ,Quien_puede_realizar_este_tramite ,Requiere_de_algun_certificado ,Detalle_certificados ,Requiere_de_un_documento_notarial ,Detalle_documentos ,Finaliza_con_la_emision_de_un_certificado ,involucra_firmaFEA ,Tipo_de_Firma_Electronica_Avanzada_FEA ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", datos2)
    #consulta_insertar_datos = f"INSERT INTO nueva_tabla ({', '.join(nombres_columnas)}) VALUES ({', '.join(['%s'] * len(nombres_columnas))})"
    #cursores[f'conexion_{j}'].executemany(consulta_insertar_datos, datos)
    conexiones[f'conexion_{j}'].commit()
    print("Datos insertados en la nueva tabla correctamente")
    #conexiones[f'conexion_{j}'].close()
    #cur2.close()

Conexión 0 creada correctamente
Tabla creada correctamente
Datos insertados en la nueva tabla correctamente
Conexión 1 creada correctamente
Tabla creada correctamente
Datos insertados en la nueva tabla correctamente
Conexión 2 creada correctamente
Tabla creada correctamente
Datos insertados en la nueva tabla correctamente
Conexión 3 creada correctamente
Tabla creada correctamente
Datos insertados en la nueva tabla correctamente
Conexión 4 creada correctamente
Tabla creada correctamente
Datos insertados en la nueva tabla correctamente


# --------------------------------------------------------------------
# --------------------------------------------------------------------

# Comparación de strings

In [59]:
def buscar_requisitos_de(tramite):
    requisitos=[]
    tramites_infos=[]
    objetivo=[]
    conexiones={}
    cursores={}
    for j in range(0, 5):
        try:
        #-----------Crea una conexión y almacénala en el diccionario--------
            conexiones[f'conexion_{j}'] = mariadb.connect(
                user="root",
                password="carnesoya02",
                host="localhost",  # Concatena el valor de j al nombre del host
                port=3313+j,  # Puerto específico de la base de datos en el equipo local
                database="schema_name"
            )
            #print(f"Conexión {j} creada correctamente")
        except mariadb.Error as e:
            print(f"Error al conectar con la base de datos {j}: {e}")
        #--------------Creación de un cursor y guardarlo en un diccionario----------
        cursores[f'conexion_{j}']=conexiones[f'conexion_{j}'].cursor()
        cursores[f'conexion_{j}'].execute("select Nombre_del_tramite, Requiere_de_algun_certificado, Detalle_certificados from nueva_tabla")
        tramites_info=cursores[f'conexion_{j}'].fetchall()
        for nombre in tramites_info:
            if nombre[0]==tramite:
                requisitos.append(nombre[2])
        break
    for j in range(0,5):
        if f'conexion_{j}' not in conexiones:
            try:
            #-----------Crea una conexión y almacénala en el diccionario--------
                conexiones[f'conexion_{j}'] = mariadb.connect(
                    user="root",
                    password="carnesoya02",
                    host="localhost",  # Concatena el valor de j al nombre del host
                    port=3313+j,  # Puerto específico de la base de datos en el equipo local
                    database="schema_name"
                )
                #print(f"Conexión {j} creada correctamente")
            except mariadb.Error as e:
                print(f"Error al conectar con la base de datos {j}: {e}")
            cursores[f'conexion_{j}']=conexiones[f'conexion_{j}'].cursor()
        cursores[f'conexion_{j}'].execute("select Nombre_del_tramite, Institucion from nueva_tabla")
        tramites_info=cursores[f'conexion_{j}'].fetchall()
        for tramit in tramites_info:
            for requisito in requisitos:
                if requisito==tramit[0]:
                    objetivo.append((tramit))
    return objetivo

In [60]:
c=buscar_requisitos_de('Inscripción en Registro de Auxiliares del Comercio de Seguros personas naturales')
print('los requisitos del tramiteInscripción en Registro de Auxiliares del Comercio de Seguros personas naturales son')
for j in c:
    print(j[0]+' de la institución '+j[1]+',')


los requisitos del tramiteInscripción en Registro de Auxiliares del Comercio de Seguros personas naturales son
Certificado de estudios para fines laborales (Ministerio de Educación) de la institución Subsecretaría de Transportes,
Certificado de antecedentes (SRCeI) de la institución Servicio de Impuestos Internos,
